In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
#The space below is used for analysis of the data from the Clean_Data directory

In [ ]:
# Eugene's codes start here

In [ ]:
# Eugene's codes end here

In [ ]:
# Feipeng's codes start here

In [ ]:
# Feipeng's codes end here

In [ ]:
# Ricardo's codes start here

x=1 
print(x)

In [ ]:
# Ricardo's codes end here

In [ ]:
# Jenny's codes start here

In [ ]:
# Jenny's codes end here